In [ ]:
from scipy.spatial import distance as dist
from imutils import face_utils
import cv2
import winsound
import dlib

In [ ]:
frequency = 2500
duration = 1000
def eyeAspectRatio(eye):
    #vertical
    A = dist.euclidean(eye[1],eye[5])
    B = dist.euclidean(eye[2],eye[4])
    #horizontal
    C = dist.euclidean(eye[0],eye[3])
    ear = (A + B) / (2.0 * C)
    return ear

In [ ]:
count = 0
earThresh = 0.3 #distance between vertical eye coordinate threshold
earFrames = 48 #consecutive frames for eye closure
shape_predictor = 'face.dat'

cam = cv2.VideoCapture(0)
detector = dlib.get_frontal_face_detector() #face detection
predictor = dlib.shape_predictor(shape_predictor) #facial landmarks

#get the coordinate of left and right eye
(l_start,l_end) = face_utils.FACIAL_LANDMARKS_IDXS['left_eye']
(r_start,r_end) = face_utils.FACIAL_LANDMARKS_IDXS['right_eye']


In [ ]:
while True:
    _,frame = cam.read()
    frame = imutils.resize(frame,width = 450)
    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    
    rects = detector(gray,0)
    for rect in rects:
        shape = predictor(gray , rect)
        shape = face_utils.shape_to_np(shape)
        
        left_eye = shape[l_start:l_end]
        right_eye = shape[r_start:r_end]
        leftEAR = eyeAspectRatio(left_eye)
        rightEAR = eyeAspectRatio(right_eye)
        
        ear = (leftEAR + rightEAR)/2.0
        
        leftEyeHull = cv2.convexHull(left_eye)
        rightEyeHull = cv2.convexHull(right_eye)
        cv2.drawContours(frame,[leftEyeHull],-1,(0,0,255))
        cv2.drawContours(frame,[rightEyeHull],-1,(0,0,255))
        
        if ear< earThresh:
            count += 1
            if count >= earFrames:
                cv2.putText(frame,"Drowsiness Detected",(10,30),cv2.FONT_HERSHEY_SIMPLEX,0.7,(0,0,255),1)
                winsound.Beep(frequency,duration)
        else:
            count = 0
    cv2.imshow('frame',frame)
    key = cv2.waitKey(10)
    if key == 27:
        break
cam.release()
cv2.destroyAllWindows()